## Deliverable 2. Create a Customer Travel Destinations Map.

In [81]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_api_key

# Configure gmaps API key
gmaps.configure(api_key=g_api_key)

In [82]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,San Juan,AR,-31.5375,-68.5364,85.51,39,34,18.14,scattered clouds
1,1,Georgetown,MY,5.4112,100.3354,80.89,84,20,4.61,few clouds
2,2,Launceston,AU,-41.4500,147.1667,51.80,74,40,3.00,scattered clouds
3,3,Mataura,NZ,-46.1927,168.8643,56.48,98,100,4.99,overcast clouds
4,4,Mount Isa,AU,-20.7333,139.5000,80.37,74,75,5.75,moderate rain


In [83]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [84]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,San Juan,AR,-31.5375,-68.5364,85.51,39,34,18.14,scattered clouds
1,1,Georgetown,MY,5.4112,100.3354,80.89,84,20,4.61,few clouds
4,4,Mount Isa,AU,-20.7333,139.5000,80.37,74,75,5.75,moderate rain
8,8,Luanda,AO,-8.8368,13.2343,77.00,83,20,11.50,few clouds
10,10,Phangnga,TH,8.4509,98.5298,86.92,82,59,1.70,broken clouds
...,...,...,...,...,...,...,...,...,...,...
683,683,Pontianak,ID,-0.0333,109.3333,76.98,94,75,2.30,mist
686,686,Cururupu,BR,-1.8283,-44.8683,80.15,82,100,6.20,light rain
687,687,Gwanda,ZW,-20.9333,29.0000,77.86,55,76,3.98,broken clouds
688,688,Benguela,AO,-12.5763,13.4055,75.78,71,85,7.14,overcast clouds


In [85]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                205
City                   205
Country                204
Lat                    205
Lng                    205
Max Temp               205
Humidity               205
Cloudiness             205
Wind Speed             205
Current Description    205
dtype: int64

In [90]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,San Juan,AR,-31.5375,-68.5364,85.51,39,34,18.14,scattered clouds
1,1,Georgetown,MY,5.4112,100.3354,80.89,84,20,4.61,few clouds
4,4,Mount Isa,AU,-20.7333,139.5000,80.37,74,75,5.75,moderate rain
8,8,Luanda,AO,-8.8368,13.2343,77.00,83,20,11.50,few clouds
10,10,Phangnga,TH,8.4509,98.5298,86.92,82,59,1.70,broken clouds
...,...,...,...,...,...,...,...,...,...,...
683,683,Pontianak,ID,-0.0333,109.3333,76.98,94,75,2.30,mist
686,686,Cururupu,BR,-1.8283,-44.8683,80.15,82,100,6.20,light rain
687,687,Gwanda,ZW,-20.9333,29.0000,77.86,55,76,3.98,broken clouds
688,688,Benguela,AO,-12.5763,13.4055,75.78,71,85,7.14,overcast clouds


In [91]:
preferred_cities_df.count()

City_ID                204
City                   204
Country                204
Lat                    204
Lng                    204
Max Temp               204
Humidity               204
Cloudiness             204
Wind Speed             204
Current Description    204
dtype: int64

In [92]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City","Country","Max Temp","Current Description","Lat","Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,San Juan,AR,85.51,scattered clouds,-31.5375,-68.5364,
1,Georgetown,MY,80.89,few clouds,5.4112,100.3354,
4,Mount Isa,AU,80.37,moderate rain,-20.7333,139.5000,
8,Luanda,AO,77.00,few clouds,-8.8368,13.2343,
10,Phangnga,TH,86.92,broken clouds,8.4509,98.5298,
11,Galesong,ID,75.22,broken clouds,-5.3166,119.3661,
14,Atuona,PF,77.67,light rain,-9.8000,-139.0333,
18,Rikitea,PF,77.22,scattered clouds,-23.1203,-134.9692,
21,Puerto Ayora,EC,82.36,scattered clouds,-0.7393,-90.3518,
23,Vila Velha,BR,80.55,broken clouds,-20.3297,-40.2925,


In [120]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_api_key
}

# 6b. Iterate through the hotel DataFrame.

for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6c. Get latitude and longitude from DataFrame
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Grab the first hotel from the results and store the name.
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [121]:
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,San Juan,AR,85.51,scattered clouds,-31.5375,-68.5364,Hotel Provincial
1,Georgetown,MY,80.89,few clouds,5.4112,100.3354,Cititel Penang
4,Mount Isa,AU,80.37,moderate rain,-20.7333,139.5000,ibis Styles Mt Isa Verona
8,Luanda,AO,77.00,few clouds,-8.8368,13.2343,EPIC SANA Luanda
10,Phangnga,TH,86.92,broken clouds,8.4509,98.5298,Sunimit Mansion
...,...,...,...,...,...,...,...
683,Pontianak,ID,76.98,mist,-0.0333,109.3333,Aston Pontianak Hotel & Convention Center
686,Cururupu,BR,80.15,light rain,-1.8283,-44.8683,POUSADA TRAVÉS'CIA
687,Gwanda,ZW,77.86,broken clouds,-20.9333,29.0000,Mafax Homeways
688,Benguela,AO,75.78,overcast clouds,-12.5763,13.4055,Hotel Praia Morena


In [122]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   204
Country                204
Max Temp               204
Current Description    204
Lat                    204
Lng                    204
Hotel Name             204
dtype: int64

In [123]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df = hotel_df.dropna()
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,San Juan,AR,85.51,scattered clouds,-31.5375,-68.5364,Hotel Provincial
1,Georgetown,MY,80.89,few clouds,5.4112,100.3354,Cititel Penang
4,Mount Isa,AU,80.37,moderate rain,-20.7333,139.5000,ibis Styles Mt Isa Verona
8,Luanda,AO,77.00,few clouds,-8.8368,13.2343,EPIC SANA Luanda
10,Phangnga,TH,86.92,broken clouds,8.4509,98.5298,Sunimit Mansion
...,...,...,...,...,...,...,...
683,Pontianak,ID,76.98,mist,-0.0333,109.3333,Aston Pontianak Hotel & Convention Center
686,Cururupu,BR,80.15,light rain,-1.8283,-44.8683,POUSADA TRAVÉS'CIA
687,Gwanda,ZW,77.86,broken clouds,-20.9333,29.0000,Mafax Homeways
688,Benguela,AO,75.78,overcast clouds,-12.5763,13.4055,Hotel Praia Morena


In [124]:
# 8a. Create the output File (CSV)
output_data_file = "vacation_search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [125]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [126]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
#heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             #max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))